In [3]:
installation de facebook scrapper
!pip install facebook-scraper
!pip install selenium==4.1.0
!pip install fake-useragent
!pip install parsel
!pip install webdriver_manager

ERROR: Could not find a version that satisfies the requirement webdriver_manage (from versions: none)
ERROR: No matching distribution found for webdriver_manage


  Using cached webdriver_manager-3.8.5-py2.py3-none-any.whl (27 kB)
  Using cached python_dotenv-0.21.1-py3-none-any.whl (19 kB)


In [4]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from parsel import Selector
from selenium.webdriver.common.keys import Keys
from facebook_scraper import get_posts
from multiprocessing import Pool, cpu_count
import re
from requests import get
import pandas as pd
import numpy as np
import time
import os
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from fake_useragent import UserAgent
import pandas as pd
import numpy as np


In [ ]:
# rechercheGoogle("mbadon",10)

In [4]:
def headless(proxy: str = "") :
        ua = UserAgent()
        userAgent = ua.random
        options = webdriver.ChromeOptions()
        options.add_argument("headless")
        options.add_argument("window-size=1500,1200")
        options.add_argument("no-sandbox")
        options.add_argument("disable-dev-shm-usage")
        options.add_argument("disable-gpu")
        options.add_argument("log-level=3")
        options.add_argument(f"user-agent={userAgent}")

        if proxy != "":
            proxy = True
            options.add_argument("proxy-server={}".format(proxy))

        driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
        return driver    


In [5]:
# initialisation de selemium
def rechercheGoogle(localite,nombrePage):
    # chargement de webdriver et du navigateur
    driver = headless()

    # aller sur facebook
    driver.get('http://www.google.com')
    # recherche le champs de saisi
    search_query = driver.find_element_by_name('q')
    search_query.send_keys('site:facebook.com AND'+'"'+localite +'"AND "'+ "maison" +'"')
    search_query.send_keys(Keys.RETURN)
    test=True
    n=0
    lienGroup=[]
    while test:
        if n<nombrePage:
            try:
            ####### recuperer les liens des differents groupes
                facebook_urls = driver.find_elements_by_xpath("//*[@class='yuRUbf']/a")
                for url in facebook_urls:
                    if "facebook" in url.get_attribute("href"):
                        for id in range(len(url.get_attribute("href").split("/"))):
                            if url.get_attribute("href").split("/")[id]=="groups":
                                lienGroup.append(url.get_attribute("href").split("/")[id+1])
                                break    
                suivantpage = driver.find_element_by_id('pnnext')
                sleep(5)
                suivantpage.click()
                n=n+1
            except:
                    test=False
                    continue
        else:
                test=False

    return lienGroup

            

In [6]:
def postGroup(nbpage,idGroup):
    n=0
    ListPost=[]
    for post in get_posts(idGroup, cookies='cookies.txt', extra_info=True, pages=nbpage, options={"comments": False}):
        print(post)
        ListPost.append(post)
ua = UserAgent()

In [ ]:
ListIdGroup=rechercheGoogle("mbadon",10)
nbpage=1
items=[(nbpage,x) for x in ListIdGroup]

In [ ]:
def multiprocessing(postGroup , list):
    with Pool(cpu_count()) as p:
        print(cpu_count())
        rec=p.starmap( postGroup,list)    
        p.terminate()
        p.join()
        return rec
# Lanceur du script
if __name__ == '__main__':
        start_time = time.time()
        list = items
        ListeDictionnaire=multiprocessing(postGroup , list)
        # decomenter la ligne suivante pour afficher les valeurs
        df = pd.concat([pd.DataFrame(lst) for lst in ListeDictionnaire])
        df.to_csv("output.csv")
        end_time = time.time() - start_time
        # Fin des traitement
        print("Temps total de compilation "+str(end_time))
      

########################################### Debut du traitement ##############################################

In [111]:
def typeMaison(publication,bag_app,bag_mais):
     dicoType={}
     count=1
     for word in publication:
        if word in bag_app:
            dicoType[count]='appartement'
            count=count+1  
        elif word in bag_mais:
            dicoType[count]='maison Individuel'
            count=count+1  
     return dicoType

In [113]:

def check(post_elmt):
    post_elmt.replace(".","")
    post_elmt = re.findall("\d+", post_elmt)
    if len(post_elmt) != 0 :
        post_elmt = post_elmt[0]  
        return post_elmt
    return 0

def montant_loyer_piece(sac_mot, post):
    dico_montant = {}
    post_suiv = ""
    cpt=0
    for i in range(0,len(post)):
        loyer=0
        if post[i] in sac_mot:
            # break
            post_pred = post[i-1]
            if i != (len(post)-1):
                post_suiv = post[i+1]
            loyer = check(post_pred)
            if loyer!=0:
                    cpt+=1
                    dico_montant[cpt]= loyer
        
            if ((loyer == 0) and (i != (len(post)-2))):
                loyer = check(post[i+1])
                if loyer!=0:
                    cpt+=1
                    dico_montant[cpt]= loyer
        
        if loyer==0:
            val=re.findall("[a-zA-Z]+", post[i])
            if len(val)!=0:
                if val[0].upper() in sac_mot:
                    loyer= check(post[i])
                    if loyer!=0:
                        cpt=cpt+1
                        dico_montant[cpt]= loyer

    return dico_montant


In [114]:
def standing(sac,eco,haut,moyen):
    dicoStanding = dict()
    k = 1
    for i in sac:
        if i in eco:
            dicoStanding[k] = "econonmique"
            k += 1
            continue
        if i in haut:
            dicoStanding[k] = "haut"
            k += 1
            continue
        if i in moyen:
            dicoStanding[k] = "moyen"
            k += 1     
    return dicoStanding

In [128]:
immeuble_sac_mot =['APPARTEMENT','APPARTEMENTS','APARTEMENT','APARTEMENTS', 
           'APPATEMENT','APPATEMENTS', 'APPAT', 'APPATS','IMMEUBLE',
           'IMMEUBLES','IMEUBLE','IMMEBLE',  'IMMEBLES','IMEBLE', 'IMEBLES','1ER','2EM','ETAGE']
villa_sac_mot=["VILLA","VILA",'VILLAS','VILAS']
piece_sac_mot=["PIECE","PIECES","PIEC",'PCS']
montant_loyer_sac_mot = ["LOYER", "FCFA","F","CFA","FR","MIL","MILLE","MILLES",'XOF','FA','€','$']
motCleLocationValide=["LOYER","LOUER","LOUE","LOU","LOCATION","LOCASSION","LOCASION","VISITE","CAUTION","CAUSION","CAUSSION","COSION","COSSION"]
motCleLocationExclu=["VENTE","VENT","TERRAIN","TERRIN","TERAIN","FONCIER","ACHETER","ACHETE"]

In [129]:
from unidecode import unidecode
import re
import codecs
def nettoyage(compltexte):
    compltexte = compltexte.replace("*", "")
    compltexte = compltexte.replace(",", "")
    compltexte = compltexte.replace(".", "")
    compltexte = re.sub(r"[^\w\s]", " ", compltexte) # supprime les caractères spéciaux et symboles
    listMot = compltexte.split() 
    listMot = [mot.upper() for mot in listMot]
    listMot = [unidecode(mot) for mot in listMot]
    return listMot

    


In [121]:

def first(postText):
    listTraite = nettoyage(postText)
    if "YOPOUGON"  in listTraite or "YOP" in listTraite:
        for mot in listTraite:
            if mot in motCleLocationValide:
                val= listTraite                
            elif mot in motCleLocationExclu:
                val=  []
            else:
                val= listTraite
    return val

        

In [ ]:

bd = pd.read_csv("data.csv")

In [122]:
publist=[]
for pub in bd["post_text"]:
  try:
    rt=first(pub)
    if len(rt) !=0:
        publist.append(rt)
  except:
        continue

In [ ]:
for publ in publist:
    typeM= typeMaison(publ,immeuble_sac_mot,villa_sac_mot)
    print("#####################")
    print(typeM)
    piece = montant_loyer_piece(piece_sac_mot,publ)
    print("#####################")
    print(piece)
    montant = montant_loyer_piece(montant_loyer_sac_mot,publ)
    print("#####################")
    print(montant)
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

    if len(typeM)==len(piece)==len(montant):
        #   on prend tous
        #   bisous   bisousbisousbisousbisousbisousbisousbisousbisousbisousbisousbisousbisousbisousbisousbisousbisousbisousbisousbisous   


In [1]:
import pandas as pd
pd.read_csv("output_ciad.csv")

,post_id,text,post_text,shared_text,original_text,time,timestamp,image,image_lowquality,images,...,image_id,image_ids,video_ids,videos,was_live,header,fetched_time,listing_title,listing_price,listing_location
0,662357315740868,POKOU immobilier & services VOUS PRÉSENTE UN N...,POKOU immobilier & services VOUS PRÉSENTE UN N...,NaN,NaN,2023-02-20 07:24:12,1.676878e+09,https://scontent.fabj4-2.fna.fbcdn.net/v/t39.3...,https://scontent.fabj4-2.fna.fbcdn.net/v/t39.3...,['https://scontent.fabj4-2.fna.fbcdn.net/v/t39...,...,NaN,[],[],[],False,Youssef Pokou‎Demarcho Maison a Louer à Abidjan☑️,2023-02-20 19:15:35.662443,NaN,NaN,NaN
1,662433745733225,Bonjour je cherche une résidence meublée pour ...,Bonjour je cherche une résidence meublée pour ...,NaN,NaN,2023-02-20 09:22:56,1.676885e+09,NaN,https://scontent.fabj4-2.fna.fbcdn.net/m1/v/t6...,[],...,NaN,[],NaN,NaN,False,Marie Josiane Ehoussou‎Demarcho Maison a Louer...,2023-02-20 19:15:39.176708,NaN,NaN,NaN
2,661730275803572,0566354162 (45mil )\n\nRécupérer à marcory ano...,0566354162 (45mil )\n\nRécupérer à marcory ano...,NaN,NaN,2023-02-19 11:45:13,1.676807e+09,https://scontent.fabj4-2.fna.fbcdn.net/v/t39.3...,https://scontent.fabj4-2.fna.fbcdn.net/v/t39.3...,['https://scontent.fabj4-2.fna.fbcdn.net/v/t39...,...,NaN,[],[],[],False,Damiba Issaka‎Demarcho Maison a Louer à Abidjan☑️,2023-02-20 19:15:59.872226,NaN,NaN,NaN
3,662830079026925,"Avec 50.000 seulement, est ce que tu sais que ...","Avec 50.000 seulement, est ce que tu sais que ...",NaN,NaN,2023-02-20 18:27:06,1.676918e+09,https://scontent.fabj4-2.fna.fbcdn.net/v/t39.3...,https://scontent.fabj4-2.fna.fbcdn.net/v/t39.3...,['https://scontent.fabj4-2.fna.fbcdn.net/v/t39...,...,NaN,[],[],[],False,So Shines‎Demarcho Maison a Louer à Abidjan☑️,2023-02-20 19:16:15.588471,NaN,NaN,NaN
4,660982819211651,Cc bradé climatiseur et placard ont va voir ow...,Cc bradé climatiseur et placard ont va voir ow...,NaN,NaN,2023-02-18 12:04:29,1.676722e+09,NaN,https://scontent.fabj4-2.fna.fbcdn.net/m1/v/t6...,[],...,NaN,[],NaN,NaN,False,Carmelle Siapoa‎Demarcho Maison a Louer à Abid...,2023-02-20 19:16:17.678044,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,1552495148566572,Boubou 15000,Boubou 15000,NaN,NaN,2023-02-20 07:40:34,1.676879e+09,NaN,https://scontent.fabj3-2.fna.fbcdn.net/v/t39.3...,NaN,...,NaN,[],NaN,NaN,False,Opounan Koné‎PRESSE IMMOBILIERE D'ABIDJAN,NaN,Boubou,15 000 FCFA,"Abidjan, Cote d'Ivoire"
542,1552625665220187,Location duplex 7 pièces + dépendance + piscin...,Location duplex 7 pièces + dépendance + piscin...,NaN,NaN,2023-02-20 12:23:49,1.676896e+09,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,False,Bénédiction Decret Divin‎PRESSE IMMOBILIERE D'...,NaN,Duplex 7 pièces + dépendance + piscine + jardi...,2 700 000 $,"Abidjan, Cote d'Ivoire"
543,5962411120532594,3 pièces tout neuf à louer a la riviera bonoum...,3 pièces tout neuf à louer a la riviera bonoum...,NaN,NaN,2023-02-18 13:15:13,1.676726e+09,NaN,https://scontent.fabj3-2.fna.fbcdn.net/v/t39.3...,NaN,...,NaN,[],NaN,NaN,False,Arsène Gnahoua‎Immo Cote d'Ivoire,NaN,3 pièces tout neuf à louer a la riviera bonoumin,250 000 $,Abidjan
544,1552808771868543,☎️:0789468538/ 0102571238\n\n📞:0546903062\n\nA...,☎️:0789468538/ 0102571238\n\n📞:0546903062\n\nA...,NaN,NaN,2023-02-20 17:14:22,1.676913e+09,NaN,https://scontent.fabj3-2.fna.fbcdn.net/v/t39.3...,NaN,...,NaN,[],NaN,NaN,False,Paolo Kangah‎PRESSE IMMOBILIERE D'ABIDJAN,NaN,Magnifique appartement de 3 pièces 2 douches a...,200 000 $,"Abidjan, Cote d'Ivoire"
